In [30]:
import sklearn.linear_model as model
import numpy as np
import pandas as pd
import pybaseball as pyb
from sklearn.metrics import r2_score

## Data Gathering

In [72]:
# Download the stats from 2024 and 2023

min_innings = 30 # change this if you want to lower the minimum number of innings (we only look at pitchers who pitched more than this number of IP in *both* seasons)

stats_2024 = pyb.pitching_stats(start_season = 2024, end_season=2024, qual=min_innings)
stats_2023 = pyb.pitching_stats(start_season = 2023, end_season=2023, qual=min_innings)

In [73]:
# Here is the list of stats I am going to look at. IDfg is an id to make sure we are looking at the same play in 2023 as in 2024. Name is the player's name. Most stats are self-explanatory, but I threw in a couple that I thought would be funny (BABIP and WPA) but are unlikely to have any coorelation. If you want to see the full list of stats, you can look at stats_2023.columns
f2023 = stats_2023.filter(items=["IDfg", "Name", "ERA", "ERA-", "FIP", "xERA", "xFIP", "SIERA", "CSW%", "K-BB%", "K%", "BB%", "WPA", "BABIP", "Zone%", "Contact%", "Swing%", "Soft%", "Hard%", "Stuff+", "Location+", "Pitching+"])

In [74]:
# Just looking at ERA and ERA- in 2024. These should both be relativly the same but just wanted to try them both. I am renaming them so I can combine this with the 2023 stats
f2024 = stats_2024.filter(items=["IDfg", "ERA", "ERA-"]).rename(columns={"ERA": "2024ERA", "ERA-": "2024ERA-"})

In [75]:
# This combines both of the tables into one big one. Note only pitchers who pitched min_innings in both seasons will be on this

all_pitching = pd.merge(f2023, f2024, "inner", "IDfg")
all_pitching

,IDfg,Name,ERA,ERA-,FIP,xERA,xFIP,SIERA,CSW%,K-BB%,K%,BB%,WPA,BABIP,Zone%,Contact%,Swing%,Soft%,Hard%,Stuff+,Location+,Pitching+,2024ERA,2024ERA-
0,10310,Zack Wheeler,3.61,82,3.15,3.18,3.54,3.53,0.275,0.220,0.269,0.050,1.94,0.292,0.421,0.740,0.511,0.214,0.313,114,107,122,2.57,62
1,13125,Gerrit Cole,2.63,61,3.16,3.48,3.60,3.63,0.277,0.212,0.270,0.058,4.39,0.261,0.425,0.765,0.499,0.151,0.311,109,106,114,3.41,85
2,12768,Sonny Gray,2.79,65,2.83,3.66,3.64,3.95,0.279,0.170,0.243,0.073,3.01,0.295,0.422,0.759,0.474,0.143,0.300,96,101,100,3.84,95
3,19291,Zac Gallen,3.47,78,3.26,4.16,3.49,3.67,0.285,0.204,0.260,0.056,2.87,0.301,0.419,0.761,0.470,0.098,0.401,97,107,105,3.65,87
4,14107,Kevin Gausman,3.16,75,2.97,3.85,3.22,3.34,0.296,0.239,0.311,0.072,2.21,0.321,0.399,0.737,0.492,0.165,0.369,101,104,107,3.83,96
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
318,14975,Chad Kuhl,8.45,195,6.78,8.79,6.35,6.00,0.248,0.016,0.166,0.150,-1.70,0.333,0.405,0.749,0.409,0.104,0.392,90,94,86,5.06,125
319,14552,Jose Ruiz,5.89,134,6.26,5.44,5.22,4.71,0.263,0.087,0.188,0.101,-0.47,0.321,0.385,0.732,0.455,0.156,0.298,98,100,100,3.71,90
320,20167,Kyle Muller,7.60,184,6.14,7.40,5.50,5.47,0.224,0.046,0.151,0.105,-2.54,0.372,0.453,0.811,0.482,0.113,0.391,93,97,91,4.01,104
321,14527,Jorge Lopez,5.95,138,5.76,5.41,4.69,4.35,0.277,0.102,0.184,0.083,-0.77,0.314,0.449,0.792,0.472,0.139,0.364,108,100,107,2.89,73


## Statistics time

In [76]:
# First, lets see how last year's ERA predicts next years ERA
era2era = model.LinearRegression().fit(X=np.array(all_pitching["ERA"]).reshape(-1,1), y=np.array(all_pitching["2024ERA"]).reshape(-1,1))

In [77]:
era_pred = era2era.predict(np.array(all_pitching["ERA"]).reshape(-1,1))

In [78]:
r2_score(np.array(all_pitching["2024ERA"]).reshape(-1,1), era_pred)

0.03214958784197497

Wow thats a small number! Lets see how every other stat does

In [79]:
y = np.array(all_pitching["2024ERA"]).reshape(-1,1)
stats = ["ERA", "ERA-", "FIP", "xERA", "xFIP", "SIERA", "CSW%", "K-BB%", "K%", "BB%", "WPA", "BABIP", "Zone%", "Contact%", "Swing%", "Soft%", "Hard%", "Stuff+", "Location+", "Pitching+"]
r2s = {}

for stat in stats:
    X = np.array(all_pitching[stat]).reshape(-1,1)
    reg = model.LinearRegression().fit(X, y)

    y_pred = reg.predict(X)
    r2 = r2_score(y, y_pred)

    print(f"{stat} has a r2 value of {r2}")
    r2s[stat] = r2


ERA has a r2 value of 0.03214958784197497
ERA- has a r2 value of 0.026493902637754196
FIP has a r2 value of 0.06497506338653403
xERA has a r2 value of 0.07413055929975554
xFIP has a r2 value of 0.05258676429994291
SIERA has a r2 value of 0.067985517652978
CSW% has a r2 value of 0.05182382411756248
K-BB% has a r2 value of 0.052696633067199294
K% has a r2 value of 0.05340438805388692
BB% has a r2 value of 0.0005676817041255777
WPA has a r2 value of 0.005295605011047089
BABIP has a r2 value of 1.043756451490907e-06
Zone% has a r2 value of 0.0024715139137491127
Contact% has a r2 value of 0.045816625300555525
Swing% has a r2 value of 0.005903968133861848
Soft% has a r2 value of 0.04680122529353048
Hard% has a r2 value of 0.031842099443893934
Stuff+ has a r2 value of 0.08070147561932994
Location+ has a r2 value of 1.000025986053199e-05
Pitching+ has a r2 value of 0.0653827349373095


So the individual stats that won are SIERA and Stuff+

Knowers of ball are probably not that surprised

Now lets see if using multiple of these stats can lead us to a formula with a higher r2

In [51]:
stats = ["SIERA", "CSW%", "K%", "BB%", "Contact%", "Swing%", "Stuff+", "Location+"]

X = all_pitching.filter(items=stats).values
y = np.array(all_pitching["2024ERA"]).reshape(-1,1)

reg = model.LinearRegression().fit(X, y)
reg.coef_, reg.intercept_

(array([[ 8.81062162e-01, -2.58419507e+00,  5.43002887e+00,
         -8.82854695e+00, -7.73513794e-01, -1.39407815e-01,
         -4.94452893e-02, -7.31716831e-03]]),
 array([6.72336924]))

In [52]:
y_pred = reg.predict(X)
r2_score(y, y_pred)

0.292406267013817

In [53]:
for i, stat in enumerate(stats):
    print(f"the coefficient of {stat} is {reg.coef_[0][i]}")

the coefficient of SIERA is 0.8810621619470753
the coefficient of CSW% is -2.5841950701528624
the coefficient of K% is 5.430028870655146
the coefficient of BB% is -8.828546945276168
the coefficient of Contact% is -0.7735137941706123
the coefficient of Swing% is -0.13940781547272943
the coefficient of Stuff+ is -0.04944528931724987
the coefficient of Location+ is -0.007317168311920412
